In [1]:
import os 
import cv2 
import PIL 
import random
from glob import glob 
import pandas as pd 
import numpy as np 

---
PAMAP2 video action data version-1 Skeleton movement generations

In [2]:
import mediapipe as mp

In [3]:
# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [4]:
def pose_estimator(pose, path):
    # create capture object
    cap = cv2.VideoCapture(path)
    if (cap.isOpened() == False): 
        print("Error reading video file")
    
    # We need to set resolutions.
    # so, convert them from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)

    skeleton_video = []
    while cap.isOpened():
        # read frame from capture object
        _, frame = cap.read()

        try:
            # convert the frame to RGB format
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # process the RGB frame to get the result
            results = pose.process(RGB)
            try:
                frame_result = results.pose_landmarks.landmark
                point_list = []
                for idx, landmark in enumerate(frame_result):
                    # print(idx, landmark.x)
                    # print(type(landmark))
                    coord = [landmark.x, landmark.y, landmark.z]
                    point_list.append(coord)
                skeleton_video.append(point_list)
            except Exception as inst:
                print(inst)
                pass
            
        except Exception as inst:
            print(inst)
            break
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

    return np.array(skeleton_video), frame_height, frame_width

In [5]:
active_points = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
point_mapping = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]


In [6]:
def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3

In [7]:
def gen_video(points, save_file, frame_h, frame_w):
    # make 3D if points are flatten 
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file, 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for frame in points:
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()

In [11]:
skel_res, fh, fw = pose_estimator(pose, '../data/knn_handpicked/10_sample/Nordic walking/1081_Nordic_walking0.mp4')

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



In [6]:
skel_res.shape

(150, 33, 3)

In [34]:
res_arr = np.hstack(np.transpose(skel_res, (1,0,2)))
res_arr.shape

(150, 99)

In [35]:
arr1 = res_arr[:50, :]

In [46]:
end_size = 60
arr_df = pd.DataFrame(data=arr1, columns=[str(i) for i in range(arr1.shape[1])])
curr_size = arr1.shape[0]
fill_points = random.sample(range(10, curr_size-10), end_size-curr_size)

df_data = []
for i, row in arr_df.iterrows():
    df_data.append(row.values)
    if i in fill_points:
        df_data.append(pd.Series(dtype='float64').values)

df_new = pd.DataFrame(df_data)
df_new = df_new.interpolate(method='linear', limit_direction='forward', axis=0)

In [50]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,0.504923,0.248967,-0.236674,0.506367,0.234039,-0.220277,0.507141,0.233005,-0.220332,0.507801,...,0.206195,0.465443,0.594284,0.115843,0.500520,0.601808,0.217059,0.448888,0.629824,0.158660
1,0.506739,0.249402,-0.280803,0.508321,0.234041,-0.259060,0.509073,0.233008,-0.259110,0.509659,...,0.163680,0.443809,0.594922,0.107995,0.564550,0.589813,0.164958,0.444948,0.640126,0.134737
2,0.509284,0.250637,-0.292833,0.510885,0.234291,-0.268654,0.511675,0.233336,-0.268708,0.512274,...,0.141871,0.449344,0.593423,0.105876,0.576062,0.586818,0.140687,0.452592,0.635396,0.132263
3,0.510013,0.253656,-0.285686,0.512187,0.236032,-0.262833,0.513212,0.235297,-0.262869,0.514056,...,0.088426,0.452018,0.595665,0.071176,0.583240,0.585148,0.088835,0.456727,0.634436,0.105958
4,0.510092,0.258336,-0.327291,0.512896,0.238881,-0.301456,0.514215,0.238255,-0.301486,0.515332,...,0.086845,0.455590,0.598130,0.063229,0.577656,0.586693,0.082139,0.456949,0.634137,0.092575
5,0.510642,0.259733,-0.296525,0.513900,0.240284,-0.275221,0.515310,0.239882,-0.275274,0.516497,...,0.087455,0.456080,0.601305,0.064729,0.584451,0.587549,0.087320,0.456078,0.633974,0.098889
6,0.510931,0.260600,-0.309159,0.514665,0.241710,-0.283483,0.516266,0.241445,-0.283502,0.517645,...,0.030775,0.467397,0.596753,0.032480,0.582334,0.590387,0.021483,0.469691,0.615883,0.059331
7,0.510900,0.261096,-0.368707,0.514832,0.242298,-0.342883,0.516497,0.242039,-0.342853,0.517928,...,0.069821,0.461949,0.601199,0.027865,0.585666,0.597202,0.063654,0.462225,0.629570,0.049948
8,0.510790,0.261958,-0.398043,0.515030,0.243259,-0.371457,0.516899,0.243029,-0.371424,0.518525,...,0.076942,0.463116,0.602037,0.032555,0.600032,0.597366,0.069746,0.461737,0.632328,0.053638
9,0.510736,0.262485,-0.353750,0.515216,0.244265,-0.328342,0.517107,0.243991,-0.328337,0.518735,...,0.061311,0.482893,0.581652,0.035378,0.608602,0.585571,0.056510,0.488888,0.604015,0.065703


In [54]:
x_cds = list(range(0, 99, 3))
y_cds = list(range(1, 99, 3))
z_cds = list(range(2, 99, 3))
print(len(x_cds), len(z_cds))

33 33


In [64]:
k = df_new.values
xyz_coods = np.transpose(np.array([k[:, x_cds], k[:, y_cds], k[:, z_cds]]), (1,2,0))[:, active_points, :]
xyz_coods.shape

(60, 12, 3)

In [68]:
gen_video(xyz_coods, '../data/pose_data/gen_extended_sub.mp4', fh, fw)

In [38]:
df_new.shape

(99, 60)

In [27]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.504923,0.506739,0.509284,0.510013,0.510092,0.510642,0.510931,0.510900,0.510790,0.510736,...,0.531290,0.532372,0.531946,0.532056,0.533472,0.535520,0.536812,0.537339,0.539650,0.543448
1,0.248967,0.249402,0.250637,0.253656,0.258336,0.259733,0.260600,0.261096,0.261958,0.262485,...,0.321489,0.320996,0.320615,0.320384,0.320204,0.319295,0.319193,0.319284,0.318336,0.317327
2,-0.236674,-0.280803,-0.292833,-0.285686,-0.327291,-0.296525,-0.309159,-0.368707,-0.398043,-0.353750,...,-0.244282,-0.284023,-0.304089,-0.375930,-0.341477,-0.353841,-0.337095,-0.373146,-0.360075,-0.355836
3,0.506367,0.508321,0.510885,0.512187,0.512896,0.513900,0.514665,0.514832,0.515030,0.515216,...,0.535406,0.536496,0.536525,0.536910,0.538234,0.539973,0.541900,0.542767,0.544444,0.547373
4,0.234039,0.234041,0.234291,0.236032,0.238881,0.240284,0.241710,0.242298,0.243259,0.244265,...,0.306534,0.306010,0.305679,0.305423,0.305240,0.303926,0.303873,0.304114,0.303229,0.301700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.601808,0.589813,0.586818,0.585148,0.586693,0.587549,0.590387,0.597202,0.597366,0.585571,...,0.624965,0.633428,0.644637,0.649158,0.648124,0.648974,0.648191,0.647308,0.646774,0.645877
95,0.217059,0.164958,0.140687,0.088835,0.082139,0.087320,0.021483,0.063654,0.069746,0.056510,...,0.256255,0.243959,0.263548,0.281221,0.310205,0.334161,0.341308,0.351859,0.362167,0.362358
96,0.448888,0.444948,0.452592,0.456727,0.456949,0.456078,0.469691,0.462225,0.461737,0.488888,...,0.454040,0.454459,0.458265,0.463609,0.461230,0.461060,0.460850,0.456319,0.456850,0.456962
97,0.629824,0.640126,0.635396,0.634436,0.634137,0.633974,0.615883,0.629570,0.632328,0.604015,...,0.638053,0.635750,0.635878,0.637820,0.637554,0.637561,0.637818,0.637979,0.638003,0.637946


In [8]:
def random_padding(arr, output_size):
    # extract and flatten interest locations
    active_locations = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28] 
    sub_arr = arr[:, active_locations, :]
    com_arr = np.hstack(np.transpose(sub_arr, (1,0,2)))

    # pad and interpolate
    arr_df = pd.DataFrame(data=com_arr, columns=[str(i) for i in range(com_arr.shape[1])])
    curr_size = arr_df.shape[0]
    fill_points = random.sample(range(0, curr_size), output_size-curr_size)

    df_data = []
    for i, row in arr_df.iterrows():
        df_data.append(row.values)
        if i in fill_points:
            df_data.append(pd.Series(dtype='float64').values)

    df_new = pd.DataFrame(df_data)
    df_new = df_new.interpolate(method='linear', limit_direction='forward', axis=0)
    return df_new.values 

In [9]:
def random_sampling(arr, output_size, skip_pre=10, skip_post=10):
    # extract and flatten interest locations
    active_locations = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28] 
    sub_arr = arr[:, active_locations, :]
    com_arr = np.hstack(np.transpose(sub_arr, (1,0,2)))

    curr_size = com_arr.shape[0]
    sample_points = random.sample(range(skip_pre, curr_size-skip_post), output_size)
    sample_points.sort()
    out_arr = com_arr[sample_points, :]
    return out_arr

In [ ]:
# generated fixed length skeleton seq. 
seq_len = 60 
all_seq = []
all_actions = []
all_ids = []

for f in glob('../data/knn_handpicked/10_sample/*/*'):
    _, action, p  = f.split('\\')
    file_id = p.split('.')[0]
    skel_res, _, _= pose_estimator(pose, f)
    cur_len = skel_res.shape[0]

    if cur_len < seq_len:
        try:
            res_seq = random_padding(skel_res, seq_len)
        except:
            res_seq = []
    elif (cur_len > seq_len) and (cur_len < seq_len + 25):
        res_seq = random_sampling(skel_res, seq_len, 0, 0)
    elif cur_len > seq_len + 25:
        res_seq = random_sampling(skel_res, seq_len, 10, 10)
    
    if res_seq != []:
        all_seq.append(res_seq)
        all_actions.append(action)
        all_ids.append(file_id)

all_arr = np.array(all_seq )

In [11]:
all_arr.shape

(180, 60, 36)

In [12]:
a,b = np.unique(all_actions, return_counts=True)
dict(zip(a,b))

{'Nordic walking': 10,
 'ascending stairs': 10,
 'car driving': 10,
 'computer work': 10,
 'cycling': 10,
 'descending stairs': 10,
 'folding laundry': 10,
 'house cleaning': 10,
 'ironing': 10,
 'lying': 10,
 'playing soccer': 10,
 'rope jumping': 10,
 'running': 10,
 'sitting': 10,
 'standing': 10,
 'vacuum cleaning': 10,
 'walking': 10,
 'watching TV': 10}

In [ ]:
all_ids

In [90]:
full_data = {'actions': all_actions, 'sequences': all_arr, 'id': all_ids}
np.savez('../data/skeleton_k10_movements.npz', all_actions, all_arr, all_ids)

In [10]:
reload_data = np.load('../data/skeleton_k10_movements.npz')
reload_data['arr_1'].shape

(180, 60, 36)

---

In [11]:
import os 
from glob import glob 
import numpy as np 
import cv2

In [12]:
def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3


In [3]:
def gen_video(points, save_file, frame_h, frame_w):
    # make 3D if points are flatten 
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file, 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for frame in points:
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()

In [13]:
reload_data = np.load('../data/skeleton_k10_movements.npz')
reload_data.files

['arr_0', 'arr_1', 'arr_2']

In [16]:
root_dir = '../data/generated_k10_skels/'
video_h, video_w = 640, 480
action_dict = {}

for i, act in enumerate(reload_data['arr_0']):
    action_dir = os.path.join(root_dir, act)
    if act in action_dict:
        action_dict[act] += 1 
    else:
        action_dict[act] = 0 
        os.mkdir(action_dir)

    file_name = reload_data['arr_2'][i]+'.mp4'
    file_dir = os.path.join(action_dir, file_name)
    gen_video(reload_data['arr_1'][i, ...], file_dir, video_h, video_w)
    